In [ ]:
# Edit the annotations to match 224x224 size to work as ground truth for crafted adversarial images.

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import skimage.io as io
import os

import json
import albumentations as A

In [2]:
image_dir = 'val2017/data/'
annotations_file = 'val2017/labels.json'

In [4]:
coco=COCO(annotations_file)

loading annotations into memory...
Done (t=0.82s)
creating index...
index created!


In [31]:
#validation set details
imgIds = coco.getImgIds()
print("Total images: {}".format(len(imgIds)))
img_id=403385
img = coco.loadImgs(img_id)[0]
print("Image example:")
print(img)

annIds=coco.getAnnIds()
print("\nTotal annotations: {}".format(len(annIds)))
ann=coco.loadAnns(coco.getAnnIds(imgIds=img['id']))
print("Annotation example:")
print(ann[0])

Total images: 5000
Image example:
{'license': 4, 'file_name': '000000403385.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000403385.jpg', 'height': 511, 'width': 640, 'date_captured': '2013-11-15 00:09:17', 'flickr_url': 'http://farm4.staticflickr.com/3526/3768289025_b29315b582_z.jpg', 'id': 403385}

Total annotations: 36781
Annotation example:
{'segmentation': [[413.39, 237.7, 414.54, 303.16, 419.13, 306.6, 412.24, 328.42, 427.17, 353.68, 411.1, 373.2, 417.99, 399.61, 440.95, 415.69, 429.47, 437.51, 429.47, 468.51, 455.88, 480.0, 481.14, 470.81, 483.44, 443.25, 474.25, 417.99, 491.48, 393.87, 498.37, 366.31, 480.0, 349.09, 494.92, 313.49, 493.78, 308.9, 498.37, 304.3, 504.11, 237.7]], 'area': 16555.1475, 'iscrowd': 0, 'image_id': 403385, 'bbox': [411.1, 237.7, 93.01, 242.3], 'category_id': 70, 'id': 1096418}


In [6]:
cats = coco.loadCats(coco.getCatIds())
print("Number of categories: {}".format(len(cats)))
nms=[cat['name'] for cat in cats]
ids=[cat['id'] for cat in cats]
print('\nCOCO categories: \n{}\n'.format(', '.join(nms)))

Number of categories: 80

COCO categories: 
person, bicycle, car, motorcycle, airplane, bus, train, truck, boat, traffic light, fire hydrant, stop sign, parking meter, bench, bird, cat, dog, horse, sheep, cow, elephant, bear, zebra, giraffe, backpack, umbrella, handbag, tie, suitcase, frisbee, skis, snowboard, sports ball, kite, baseball bat, baseball glove, skateboard, surfboard, tennis racket, bottle, wine glass, cup, fork, knife, spoon, bowl, banana, apple, sandwich, orange, broccoli, carrot, hot dog, pizza, donut, cake, chair, couch, potted plant, bed, dining table, toilet, tv, laptop, mouse, remote, keyboard, cell phone, microwave, oven, toaster, sink, refrigerator, book, clock, vase, scissors, teddy bear, hair drier, toothbrush



In [7]:
category_id_to_name = dict(zip(ids, nms))
print(category_id_to_name)

{1: 'person', 2: 'bicycle', 3: 'car', 4: 'motorcycle', 5: 'airplane', 6: 'bus', 7: 'train', 8: 'truck', 9: 'boat', 10: 'traffic light', 11: 'fire hydrant', 13: 'stop sign', 14: 'parking meter', 15: 'bench', 16: 'bird', 17: 'cat', 18: 'dog', 19: 'horse', 20: 'sheep', 21: 'cow', 22: 'elephant', 23: 'bear', 24: 'zebra', 25: 'giraffe', 27: 'backpack', 28: 'umbrella', 31: 'handbag', 32: 'tie', 33: 'suitcase', 34: 'frisbee', 35: 'skis', 36: 'snowboard', 37: 'sports ball', 38: 'kite', 39: 'baseball bat', 40: 'baseball glove', 41: 'skateboard', 42: 'surfboard', 43: 'tennis racket', 44: 'bottle', 46: 'wine glass', 47: 'cup', 48: 'fork', 49: 'knife', 50: 'spoon', 51: 'bowl', 52: 'banana', 53: 'apple', 54: 'sandwich', 55: 'orange', 56: 'broccoli', 57: 'carrot', 58: 'hot dog', 59: 'pizza', 60: 'donut', 61: 'cake', 62: 'chair', 63: 'couch', 64: 'potted plant', 65: 'bed', 67: 'dining table', 70: 'toilet', 72: 'tv', 73: 'laptop', 74: 'mouse', 75: 'remote', 76: 'keyboard', 77: 'cell phone', 78: 'micro

In [42]:
# Open original data
print(annotations_file)
f = open('val2017/labels.json', 'r')
data = json.load(f)
f.close()

val2017/labels.json


In [43]:
#check the data
for key in data['images']:
    print(key)
    file_name = key['file_name']
    key_id = key['id']
    coco_url = key['coco_url']
    width = key['width']
    height = key['height']
    print('File name:', key['file_name'])
    print('ID:', key['id'])
    print('URL:', key['coco_url'])
    print('Width:', key['width'],'Height:',key['height'], '\n')
    bboxes = []
    category_ids = []
    class_labels = []
    for annotation in data['annotations']:
        if annotation['image_id'] == key_id:
            bboxes.append(annotation['bbox'])
            category_ids.append(annotation['category_id'])
            class_labels.append(category_id_to_name[annotation['category_id']])
            print(annotation['bbox'], annotation['category_id'], category_id_to_name[annotation['category_id']])
    print('\n', 'Bounding boxes:', bboxes, '\n')
    print('Category ids:',category_ids, '\n')
    print('Class labels:',class_labels, '\n')
    image_path = 'val2017/data/' + file_name
    image = io.imread(image_path)
    print(image.shape)
    break

{'license': 4, 'file_name': '000000397133.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000397133.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-14 17:02:52', 'flickr_url': 'http://farm7.staticflickr.com/6116/6255196340_da26cf2c9e_z.jpg', 'id': 397133}
File name: 000000397133.jpg
ID: 397133
URL: http://images.cocodataset.org/val2017/000000397133.jpg
Width: 640 Height: 427 

[217.62, 240.54, 38.99, 57.75] 44 bottle
[1.0, 240.24, 346.63, 186.76] 67 dining table
[388.66, 69.92, 109.41, 277.62] 1 person
[135.57, 249.43, 22.32, 28.79] 49 knife
[31.28, 344.0, 68.12, 40.83] 51 bowl
[59.63, 287.36, 76.07, 41.3] 51 bowl
[1.36, 164.33, 192.56, 98.37] 79 oven
[0.0, 262.81, 62.16, 36.77] 1 person
[119.4, 272.51, 24.82, 34.25] 47 cup
[141.47, 267.91, 32.19, 35.86] 47 cup
[155.97, 168.95, 26.03, 17.13] 51 bowl
[157.2, 114.15, 17.86, 15.82] 51 bowl
[98.75, 304.78, 10.78, 5.57] 56 broccoli
[166.03, 256.36, 8.82, 18.58] 50 spoon
[86.41, 293.97, 23.96, 11.18] 56 broccoli
[70.

In [45]:
def resize_image(img_arr, bboxes, h, w, class_labels):
    """
    :param img_arr: original image as a numpy array
    :param bboxes: bboxes as numpy array where each row is 'x_min', 'y_min', 'x_max', 'y_max', "class_id"
    :param h: resized height dimension of image
    :param w: resized weight dimension of image
    :return: dictionary containing {image:transformed, bboxes:['x_min', 'y_min', 'x_max', 'y_max', "class_id"]}
    """
    # create resize transform pipeline
    transform = A.Compose(
        [A.Resize(height=h, width=w, always_apply=True)],
        bbox_params=A.BboxParams(format='coco', label_fields=['class_labels']))
    
    #print(bboxes)
    transformed = transform(image=img_arr, bboxes=bboxes, class_labels=class_labels)
    transformed_bboxes = transformed['bboxes']
    
    bounding_boxes_coco = []
    for i in transformed_bboxes:
        i = list(np.around(i, decimals=2))
        bounding_boxes_coco.append(i)
    
    return bounding_boxes_coco

In [40]:
#new_bboxes = resize_image(image, bboxes, new_height, new_width, class_labels)
#new_bboxes = resize_image(image, bboxes, height, width, class_labels)
#print(bboxes)
#print(new_bboxes)

In [46]:
new_height, new_width = 224, 224
a = 0
# Make changes into dictionary
for key in data['images']:
    #print(key)
    a +=1
    file_name = key['file_name']
    print(a,'.', file_name)
    key_id = key['id']
    image_path = 'val2017/data/' + file_name
    image = io.imread(image_path)
    for annotation in data['annotations']:
        if annotation['image_id'] == key['id']:
            bboxes = []
            category_ids = []
            class_labels = []
            bboxes.append(annotation['bbox'])
            category_ids.append(annotation['category_id'])
            class_labels.append(category_id_to_name[annotation['category_id']])
            new_bboxes = resize_image(image, bboxes, new_height, new_width, class_labels)
            annotation['bbox'] = new_bboxes[0]
            annotation['segmentation'] = []
    key['width'] = 224
    key['height'] = 224
    #break

1 . 000000397133.jpg
2 . 000000037777.jpg
3 . 000000252219.jpg
4 . 000000087038.jpg
5 . 000000174482.jpg
6 . 000000403385.jpg
7 . 000000006818.jpg
8 . 000000480985.jpg
9 . 000000458054.jpg
10 . 000000331352.jpg
11 . 000000296649.jpg
12 . 000000386912.jpg
13 . 000000502136.jpg
14 . 000000491497.jpg
15 . 000000184791.jpg
16 . 000000348881.jpg
17 . 000000289393.jpg
18 . 000000522713.jpg
19 . 000000181666.jpg
20 . 000000017627.jpg
21 . 000000143931.jpg
22 . 000000303818.jpg
23 . 000000463730.jpg
24 . 000000460347.jpg
25 . 000000322864.jpg
26 . 000000226111.jpg
27 . 000000153299.jpg
28 . 000000308394.jpg
29 . 000000456496.jpg
30 . 000000058636.jpg
31 . 000000041888.jpg
32 . 000000184321.jpg
33 . 000000565778.jpg
34 . 000000297343.jpg
35 . 000000336587.jpg
36 . 000000122745.jpg
37 . 000000219578.jpg
38 . 000000555705.jpg
39 . 000000443303.jpg
40 . 000000500663.jpg
41 . 000000418281.jpg
42 . 000000025560.jpg
43 . 000000403817.jpg
44 . 000000085329.jpg
45 . 000000329323.jpg
46 . 000000239274.j

369 . 000000173044.jpg
370 . 000000168458.jpg
371 . 000000512194.jpg
372 . 000000370042.jpg
373 . 000000189436.jpg
374 . 000000533958.jpg
375 . 000000117645.jpg
376 . 000000221708.jpg
377 . 000000202228.jpg
378 . 000000403565.jpg
379 . 000000211042.jpg
380 . 000000492878.jpg
381 . 000000441586.jpg
382 . 000000547816.jpg
383 . 000000306733.jpg
384 . 000000530099.jpg
385 . 000000312278.jpg
386 . 000000097679.jpg
387 . 000000564127.jpg
388 . 000000251065.jpg
389 . 000000003845.jpg
390 . 000000138819.jpg
391 . 000000205834.jpg
392 . 000000348708.jpg
393 . 000000166521.jpg
394 . 000000485802.jpg
395 . 000000099054.jpg
396 . 000000022969.jpg
397 . 000000570539.jpg
398 . 000000278353.jpg
399 . 000000158548.jpg
400 . 000000461405.jpg
401 . 000000176606.jpg
402 . 000000044699.jpg
403 . 000000559956.jpg
404 . 000000268996.jpg
405 . 000000011197.jpg
406 . 000000483667.jpg
407 . 000000448810.jpg
408 . 000000000724.jpg
409 . 000000051961.jpg
410 . 000000375278.jpg
411 . 000000302165.jpg
412 . 00000

731 . 000000497599.jpg
732 . 000000455352.jpg
733 . 000000236914.jpg
734 . 000000232684.jpg
735 . 000000415238.jpg
736 . 000000290843.jpg
737 . 000000519522.jpg
738 . 000000144784.jpg
739 . 000000167486.jpg
740 . 000000392228.jpg
741 . 000000488673.jpg
742 . 000000191013.jpg
743 . 000000080057.jpg
744 . 000000570169.jpg
745 . 000000224807.jpg
746 . 000000163562.jpg
747 . 000000136355.jpg
748 . 000000492362.jpg
749 . 000000102707.jpg
750 . 000000232563.jpg
751 . 000000010977.jpg
752 . 000000051598.jpg
753 . 000000032285.jpg
754 . 000000520910.jpg
755 . 000000131273.jpg
756 . 000000206411.jpg
757 . 000000472375.jpg
758 . 000000481404.jpg
759 . 000000471991.jpg
760 . 000000017436.jpg
761 . 000000177934.jpg
762 . 000000165518.jpg
763 . 000000571718.jpg
764 . 000000459467.jpg
765 . 000000135673.jpg
766 . 000000134886.jpg
767 . 000000485895.jpg
768 . 000000287545.jpg
769 . 000000577182.jpg
770 . 000000289222.jpg
771 . 000000372819.jpg
772 . 000000310072.jpg
773 . 000000087144.jpg
774 . 00000

1089 . 000000104669.jpg
1090 . 000000439994.jpg
1091 . 000000430377.jpg
1092 . 000000512776.jpg
1093 . 000000095155.jpg
1094 . 000000184978.jpg
1095 . 000000199055.jpg
1096 . 000000431848.jpg
1097 . 000000333772.jpg
1098 . 000000128699.jpg
1099 . 000000121591.jpg
1100 . 000000176799.jpg
1101 . 000000424521.jpg
1102 . 000000254016.jpg
1103 . 000000523807.jpg
1104 . 000000073946.jpg
1105 . 000000230819.jpg
1106 . 000000082715.jpg
1107 . 000000085195.jpg
1108 . 000000435299.jpg
1109 . 000000050828.jpg
1110 . 000000027696.jpg
1111 . 000000062808.jpg
1112 . 000000497344.jpg
1113 . 000000361147.jpg
1114 . 000000541123.jpg
1115 . 000000163611.jpg
1116 . 000000010707.jpg
1117 . 000000409630.jpg
1118 . 000000343706.jpg
1119 . 000000199395.jpg
1120 . 000000514797.jpg
1121 . 000000486104.jpg
1122 . 000000514586.jpg
1123 . 000000279774.jpg
1124 . 000000474078.jpg
1125 . 000000000872.jpg
1126 . 000000032038.jpg
1127 . 000000261732.jpg
1128 . 000000012120.jpg
1129 . 000000346638.jpg
1130 . 000000306

1431 . 000000105923.jpg
1432 . 000000325031.jpg
1433 . 000000166664.jpg
1434 . 000000166642.jpg
1435 . 000000143961.jpg
1436 . 000000304545.jpg
1437 . 000000212573.jpg
1438 . 000000225670.jpg
1439 . 000000575357.jpg
1440 . 000000186422.jpg
1441 . 000000319369.jpg
1442 . 000000026941.jpg
1443 . 000000252716.jpg
1444 . 000000064868.jpg
1445 . 000000205324.jpg
1446 . 000000447313.jpg
1447 . 000000172977.jpg
1448 . 000000560880.jpg
1449 . 000000449406.jpg
1450 . 000000491613.jpg
1451 . 000000048555.jpg
1452 . 000000061584.jpg
1453 . 000000244099.jpg
1454 . 000000460682.jpg
1455 . 000000481159.jpg
1456 . 000000028809.jpg
1457 . 000000050679.jpg
1458 . 000000010764.jpg
1459 . 000000537827.jpg
1460 . 000000329827.jpg
1461 . 000000161820.jpg
1462 . 000000116362.jpg
1463 . 000000500270.jpg
1464 . 000000308165.jpg
1465 . 000000125245.jpg
1466 . 000000544444.jpg
1467 . 000000378244.jpg
1468 . 000000090956.jpg
1469 . 000000313562.jpg
1470 . 000000258388.jpg
1471 . 000000273711.jpg
1472 . 000000358

...

3508 . 000000262048.jpg
3509 . 000000519208.jpg
3510 . 000000270402.jpg
3511 . 000000250127.jpg
3512 . 000000549930.jpg
3513 . 000000112626.jpg
3514 . 000000000285.jpg
3515 . 000000573391.jpg
3516 . 000000447200.jpg
3517 . 000000261036.jpg
3518 . 000000269932.jpg
3519 . 000000426297.jpg
3520 . 000000068933.jpg
3521 . 000000047571.jpg
3522 . 000000023126.jpg
3523 . 000000242678.jpg
3524 . 000000229948.jpg
3525 . 000000451879.jpg
3526 . 000000158660.jpg
3527 . 000000059598.jpg
3528 . 000000485480.jpg
3529 . 000000006012.jpg
3530 . 000000013201.jpg
3531 . 000000243989.jpg
3532 . 000000044260.jpg
3533 . 000000581357.jpg
3534 . 000000476119.jpg
3535 . 000000287667.jpg
3536 . 000000527784.jpg
3537 . 000000179898.jpg
3538 . 000000349152.jpg
3539 . 000000187055.jpg
3540 . 000000493334.jpg
3541 . 000000318455.jpg
3542 . 000000520659.jpg
3543 . 000000465430.jpg
3544 . 000000209829.jpg
3545 . 000000329080.jpg
3546 . 000000190307.jpg
3547 . 000000200421.jpg
3548 . 000000551820.jpg
3549 . 000000254

3851 . 000000094871.jpg
3852 . 000000015746.jpg
3853 . 000000390301.jpg
3854 . 000000039484.jpg
3855 . 000000273617.jpg
3856 . 000000288685.jpg
3857 . 000000076261.jpg
3858 . 000000176847.jpg
3859 . 000000542073.jpg
3860 . 000000494863.jpg
3861 . 000000307598.jpg
3862 . 000000505942.jpg
3863 . 000000309173.jpg
3864 . 000000304365.jpg
3865 . 000000014473.jpg
3866 . 000000396338.jpg
3867 . 000000416330.jpg
3868 . 000000014831.jpg
3869 . 000000572408.jpg
3870 . 000000125806.jpg
3871 . 000000090062.jpg
3872 . 000000512648.jpg
3873 . 000000505573.jpg
3874 . 000000160772.jpg
3875 . 000000088250.jpg
3876 . 000000441543.jpg
3877 . 000000293300.jpg
3878 . 000000511398.jpg
3879 . 000000093965.jpg
3880 . 000000163290.jpg
3881 . 000000572900.jpg
3882 . 000000134856.jpg
3883 . 000000267933.jpg
3884 . 000000040036.jpg
3885 . 000000576955.jpg
3886 . 000000185473.jpg
3887 . 000000465585.jpg
3888 . 000000081738.jpg
3889 . 000000442463.jpg
3890 . 000000450399.jpg
3891 . 000000516173.jpg
3892 . 000000224

4200 . 000000405432.jpg
4201 . 000000314251.jpg
4202 . 000000260657.jpg
4203 . 000000290619.jpg
4204 . 000000214869.jpg
4205 . 000000256192.jpg
4206 . 000000041872.jpg
4207 . 000000208208.jpg
4208 . 000000098520.jpg
4209 . 000000388258.jpg
4210 . 000000026204.jpg
4211 . 000000296317.jpg
4212 . 000000234526.jpg
4213 . 000000202339.jpg
4214 . 000000238039.jpg
4215 . 000000459437.jpg
4216 . 000000355817.jpg
4217 . 000000351530.jpg
4218 . 000000569273.jpg
4219 . 000000357748.jpg
4220 . 000000020107.jpg
4221 . 000000415536.jpg
4222 . 000000423617.jpg
4223 . 000000084170.jpg
4224 . 000000564336.jpg
4225 . 000000509258.jpg
4226 . 000000104119.jpg
4227 . 000000042563.jpg
4228 . 000000066135.jpg
4229 . 000000027932.jpg
4230 . 000000488710.jpg
4231 . 000000363072.jpg
4232 . 000000469246.jpg
4233 . 000000234660.jpg
4234 . 000000430871.jpg
4235 . 000000140203.jpg
4236 . 000000434996.jpg
4237 . 000000018833.jpg
4238 . 000000231580.jpg
4239 . 000000350003.jpg
4240 . 000000409867.jpg
4241 . 000000033

4544 . 000000285047.jpg
4545 . 000000215644.jpg
4546 . 000000014007.jpg
4547 . 000000537672.jpg
4548 . 000000076211.jpg
4549 . 000000242411.jpg
4550 . 000000376478.jpg
4551 . 000000080413.jpg
4552 . 000000218249.jpg
4553 . 000000122166.jpg
4554 . 000000543047.jpg
4555 . 000000066231.jpg
4556 . 000000122217.jpg
4557 . 000000447169.jpg
4558 . 000000129322.jpg
4559 . 000000507473.jpg
4560 . 000000565391.jpg
4561 . 000000229659.jpg
4562 . 000000545958.jpg
4563 . 000000524850.jpg
4564 . 000000116479.jpg
4565 . 000000457078.jpg
4566 . 000000131386.jpg
4567 . 000000469174.jpg
4568 . 000000288882.jpg
4569 . 000000115946.jpg
4570 . 000000436738.jpg
4571 . 000000226802.jpg
4572 . 000000291664.jpg
4573 . 000000340272.jpg
4574 . 000000397639.jpg
4575 . 000000566923.jpg
4576 . 000000148508.jpg
4577 . 000000404601.jpg
4578 . 000000326627.jpg
4579 . 000000440508.jpg
4580 . 000000396568.jpg
4581 . 000000079408.jpg
4582 . 000000286507.jpg
4583 . 000000023937.jpg
4584 . 000000261706.jpg
4585 . 000000350

4888 . 000000554266.jpg
4889 . 000000008629.jpg
4890 . 000000161925.jpg
4891 . 000000064898.jpg
4892 . 000000547854.jpg
4893 . 000000033005.jpg
4894 . 000000388215.jpg
4895 . 000000111609.jpg
4896 . 000000064718.jpg
4897 . 000000546325.jpg
4898 . 000000082688.jpg
4899 . 000000367569.jpg
4900 . 000000209753.jpg
4901 . 000000531135.jpg
4902 . 000000447187.jpg
4903 . 000000311002.jpg
4904 . 000000458045.jpg
4905 . 000000246436.jpg
4906 . 000000440184.jpg
4907 . 000000085157.jpg
4908 . 000000256916.jpg
4909 . 000000504415.jpg
4910 . 000000246522.jpg
4911 . 000000446703.jpg
4912 . 000000367095.jpg
4913 . 000000529148.jpg
4914 . 000000480212.jpg
4915 . 000000442993.jpg
4916 . 000000071938.jpg
4917 . 000000136466.jpg
4918 . 000000028452.jpg
4919 . 000000581482.jpg
4920 . 000000217425.jpg
4921 . 000000140439.jpg
4922 . 000000151820.jpg
4923 . 000000260105.jpg
4924 . 000000127394.jpg
4925 . 000000238410.jpg
4926 . 000000166918.jpg
4927 . 000000290768.jpg
4928 . 000000357567.jpg
4929 . 000000237

In [47]:
# Save modified data to another file
json_data = json.dumps(data)

f = open('val2017/labels_copy.json',"w")
f.write(json_data)
f.close()

In [51]:
# check modified file
f = open('val2017/labels_copy.json', 'r')
edited_data = json.load(f)
f.close()

for key in edited_data['images']:
    #print(key)
    #print('File name:', key['file_name'])
    #print('ID:', key['id'])
    #print('URL:', key['coco_url'])
    #print('Width:', key['width'],'Height:',key['height'])
    if key['width'] != 224 or key['height'] != 224:
        print(print('size incorrect:', key['file_name']))
        break
    for annotation in edited_data['annotations']:
        if annotation['image_id'] == key['id']:
            #print(annotation['bbox'])
            #print(annotation['category_id'], category_id_to_name[annotation['category_id']])
            #print(annotation['segmentation'])
            if annotation['segmentation'] != []:
                print('segmentation not empty', key['file_name'])
                break
    #break

In [ ]:
#A.convert_bbox_from_albumentations (bbox, target_format, rows, cols, check_validity=False)